In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error

from data_loader import *
from stats import *

## Settings

In [ ]:
num_train = 60
num_test = 20
num_total = num_train + num_test

window_size = 10
kernel_size = 5

tickers = [
    "AMZN", "TSLA", "AAPL", "NVDA", "AMGN", 
    "AAL", "LUV", "JNJ", "UNH", "JEPI"
]


writer = SummaryWriter(log_dir=f"runs/sparsely_connected/{datetime.now().strftime('%Y%m%d_%H%M%S')}")


## Data

In [ ]:
X = []
y = []

for ticker in tickers:
    X1, y1 = load_stock(ticker)
    X.append(X1)
    y.append(y1)

X = np.stack(X)
y = np.stack(y)


X_train, y_train = setup_cov_tensors(X[..., -num_total: -num_test], y[..., -num_total: -num_test], window_size)
X_test, y_test = setup_cov_tensors(X[..., -num_test-window_size:], y[..., -num_test-window_size:], window_size)


In [ ]:
print(X_train.shape)
print(y_train.shape)

## Model

In [ ]:
class MultiStockReturnPredictor(nn.Module):
    def __init__(self, per_stock_features, kernel_size, num_outputs, sequence_length):
        super(MultiStockReturnPredictor, self).__init__()
        
        self.per_stock_features = per_stock_features
        convolution_channels = 18
        convolution_out = 9
        self.num_stocks = num_outputs  # Number of stocks
        
        # Dynamically create convolutional layers for each stock
        self.conv1 = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels=per_stock_features, out_channels=convolution_channels, kernel_size=kernel_size, stride=1, padding=0),
                nn.Tanh(),
                nn.Conv1d(in_channels=convolution_channels, out_channels=convolution_out, kernel_size=kernel_size, stride=1, padding=0),
                nn.Tanh()
            )
            for _ in range(self.num_stocks)
        ])
        
        # Fully connected layers
        length_after_conv1 = sequence_length - kernel_size + 1
        length_after_conv2 = length_after_conv1 - kernel_size + 1
        fc1_input_size = self.num_stocks * convolution_out * length_after_conv2

        self.fc1 = nn.Linear(fc1_input_size, 6)
        self.fc2 = nn.Linear(6, num_outputs)

        # Initialize weights
        self.apply(self.initialize_weights)

    def initialize_weights(self, module):
        if isinstance(module, nn.Conv1d):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

    def exponential_smoothing(self, x, alpha=0.3):
        # Apply exponential smoothing along the time dimension
        x_np = x.detach().cpu().numpy()
        smoothed = pd.DataFrame(x_np).ewm(alpha=alpha, axis=2).mean().values
        return torch.tensor(smoothed, dtype=torch.float32, device=x.device)

    def forward(self, x):
        # x shape: [batch_size, num_stocks, per_stock_features, sequence_length]
        stock_outputs = []

        # Apply convolutional layers for each stock
        for i in range(self.num_stocks):
            stock_x = x[:, i, :, :] 
            stock_out = self.conv1[i](stock_x)  # Pass through stock-specific convolutional layers
            stock_outputs.append(stock_out)

        # Concatenate outputs from all stocks
        x_combined = torch.cat(stock_outputs, dim=1) 
        x_combined = x_combined.view(x_combined.size(0), -1)

        # Fully connected layers
        x = torch.tanh(self.fc1(x_combined))
        x = self.fc2(x)

        return x


## Train

In [ ]:
model = MultiStockReturnPredictor(X.shape[1], kernel_size, y.shape[0], window_size)

# if cuda:
#     model.cuda()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)


for epoch in range(4000):
    train(epoch, X_train, y_train, model, optimizer, criterion, writer)
    if epoch % 100 == 0:
        pred, obj = test(epoch, X_test, y_test, model, writer)

writer.close()


## Plot

In [ ]:
df = compute_prediction_stats(tickers, pred, obj)
df

In [ ]:
stock_id = 0
_pred = pred[:, stock_id]
_obj = obj[:, stock_id]

obj_sort = np.argsort(_obj)
plt.plot(np.cumsum(_pred[obj_sort]), label="pred")
plt.plot(np.cumsum(_obj[obj_sort]), label="obj")

plt.legend()

In [ ]:
plt.plot((_pred), label="pred")
plt.plot((_obj), label="obj")

plt.legend()